# Making Networks from Google Dataset

Based on Michael's code, can be more easily dealt with here (converted from Julia to python)

In [11]:
import pandas as pd
import os
import math
import numpy as np
import csv
import time

In [12]:
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
    display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
## Insert whatever audio file you want above

In [13]:
we = os.listdir('/Volumes/HardDrive/weekbyweek/')
weekbyweek = []
for item in we:
    if '._' not in item:
        weekbyweek.append(item)

In [14]:
weekbyweek = sorted(weekbyweek)

In [15]:
weeks = len(weekbyweek)

In [16]:
def lineinbox(x):
    truth = True
    box1 = [112, 129, -45, -12]
    # Is the starting point of the line in the box?
    if (box1[2] < x[2] < box1[3]) & (box1[0]< x[3] < box1[1]):
        # Is the end point of the line in the box?
        if (box1[2] < x[5] < box1[3]) & (box1[0]< x[6] < box1[1]):
            truth = False
    return truth

In [17]:
def inbox(df):
    inds = []
    for row in df.itertuples():
        if lineinbox(list(row[1:])):
            inds.append(row[0])
    df = df.drop(inds)
    return df

In [18]:
def retrieve_posn(df):
    label_town_dict = {}
    
    labels = list(set(list(df[1]) + list(df[4]) ))
    
    starting = time.time()
    
    positions = []
    for l in range(len(labels)):
        lab = labels[l]
#         try:
#             temp_sou = df.loc[df[1] == lab]
#             lats,lons = list(temp_sou[2])[0], list(temp_sou[3])[0]
#         except:
#             temp_sou = df.loc[df[4] == lab]
#             lats,lons = list(temp_sou[5])[0], list(temp_sou[6])[0]
        
#         try:
#             temp_des = df.loc[df[4] == lab]
#             latd,lond = list(temp_des[5])[0], list(temp_des[6])[0]
#         except:
#             temp_des = df.loc[df[1] == lab]
#             latd,lond = list(temp_des[2])[0], list(temp_des[3])[0]
        
#         if (lats == latd) & (lons == lond):
        if lab not in label_town_dict:
            temp_df = df.loc[df[1] == lab]
            if len(temp_df) == 0:
                temp_df = df.loc[df[4] == lab]
                lats,lons = list(temp_df[5])[0], list(temp_df[6])[0]
                ind, dist = nearesttown(lons,lats, town_posn)
                label_town_dict[lab] = towns[ind]
            else:
                lats,lons = list(temp_df[2])[0], list(temp_df[3])[0]
                ind, dist = nearesttown(lons,lats, town_posn)
                label_town_dict[lab] = towns[ind]
        
            #positions.append([lats,lons])
        #else:
        #    print('Problem!')
    
    return labels, label_town_dict

In [19]:
def gc_dist(pos1,pos2):
    # Calculte geodesic distance between two positions
    
    pi180 = math.pi/180
    phi1 = pos1[1] * pi180
    phi2 = pos2[1] * pi180
    the1 = pos1[0] * pi180
    the2 = pos2[0] * pi180
    
    d = (math.sin(phi2 - phi1)/2)**2 + math.cos(phi1)*math.cos(phi2)*(math.sin((the2-the1)/2))**2
    d = math.asin(math.sqrt(d))
    d = 2*6378*d #Earth radius is 6378 km
    d +=1
    
    return d

In [20]:
def nearesttown(long,lat,posn):
    dist = 100
    ind = 0
    for i in range(len(posn)):
        dd = gc_dist([long,lat],posn[i])
        # Making Perth's catchment area bigger than that of the towns around it
        if i == 0:
            if dd < 60:
                dist = 0 # This means no other town can beat it 
                ind = i 
        # For all other points, find the closest town
        if dd < dist:
            dist = dd
            ind = i
    return ind, dist

In [44]:
wapop = pd.read_csv('/Users/Yuval/Desktop/Thesis/all-wa.csv')
waloc = pd.read_csv('/Users/Yuval/Desktop/Thesis/all-wa-pos.csv')

In [45]:
#majorlocalities = 58
#brk = majorlocalities*2
#global scope intended- this accounts for the way the all-wa.csv database is structured - the first 58 towns get 
#given two lines, and the rest get given one (this is becuase it is actually two different databases stapled together.
# Hence brk is when the break should happen.
#wapop = wapop.drop([2*i + 1 for i in range(majorlocalities)])
#wapop = wapop.reset_index()
#wapop.drop(['index'],axis = 1)

In [46]:
waloc = waloc.loc[waloc['Population'] >0]

In [47]:
# When doing big towns
waloc = waloc[:58]

In [48]:
# When ignoring sattelite towns
waloc = waloc.loc[(waloc['City / Town'] != 'Yanchep') & (waloc['City / Town'] != 'Bullsbrook') &
                  (waloc['City / Town'] != 'Mundijong') & (waloc['City / Town'] != 'Serpentine') &
                 (waloc['City / Town'] != 'Two Rocks') ]
waloc = waloc.reset_index()
waloc = waloc.drop(columns = ['index'])

In [50]:
town_posn = list(zip(list(waloc.iloc[:,2]),list(waloc.iloc[:,3])))
towns = list(waloc['City / Town'])

In [51]:
len(towns)

53

In [52]:
try:
    for i in range(len(weekbyweek)):
        print(i)
        start = time.time()

        name = weekbyweek[i]

        df = pd.read_csv('/Volumes/HardDrive/weekbyweek/' + name, header = None)
        df = inbox(df)

        labels, label_town_dict = retrieve_posn(df)
        df[1] = list(df[1].replace(label_town_dict))
        df[4] = list(df[4].replace(label_town_dict))

    #     for row in df.itertuples():
    #         row[1] = label_town_dict[row[1]]
    #         row[4] = label_town_dict[row[4]]

        town_flow = []
        for source in towns:
            ts = []
            for destination in towns:
                if source != destination: #ignore self loops for now
                    t_df = df.loc[df[1] == source]
                    t_df = t_df.loc[t_df[4] == destination]
                    ts.append(sum([math.exp(i) for i in list(t_df[7])]))
                else:
                    ts.append(0)
            town_flow.append(ts)

        with open("/Volumes/HardDrive/New_Workflow/53_towns_network/Town_Flow_" + name[23:-4] + ".csv", "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerows(town_flow)

        finish = time.time()

        print(i, finish-start)
except:
    allDone()

0
0 26.213642835617065
1
1 25.01914119720459
2
2 24.450532913208008
3
3 30.66529607772827
4
4 25.651282787322998
5
5 26.87735104560852
6
6 35.38549995422363
7
7 30.28413486480713
8
8 36.66628098487854
9
9 32.52827596664429
10
10 39.10962915420532
11
11 34.796810150146484
12
12 36.25009274482727
13
13 41.651654958724976
14
14 35.83808398246765
15
15 36.04962611198425
16
16 35.98471713066101
17
17 41.62951135635376
18
18 36.5116229057312
19
19 44.72113513946533
20
20 40.06819009780884
21
21 45.69853115081787
22
22 39.832895040512085
23
23 44.70046401023865
24
24 40.18725085258484
25
25 45.01520609855652
26
26 40.12059497833252
27
27 40.41321015357971
28
28 40.16097712516785
29
29 39.9349570274353
30
30 45.181132078170776
31
31 40.05910611152649
32
32 39.25246620178223
33
33 38.74809718132019
34
34 39.17713499069214
35
35 38.89863872528076
36
36 39.6530179977417
37
37 40.38354301452637
38
38 43.32791996002197
39
39 38.7989981174469
40
40 37.36222314834595
41
41 37.36045694351196
42
42 36.

In [53]:
allDone()

In [247]:
df = pd.read_csv("/Volumes/HardDrive/New_Workflow/53_towns_network/Town_Flow_" + name[23:-4] + ".csv", header = None)

In [248]:
## MAKE SURE TO CHECK THE DIMENSIONS ARE CORRECT THIS TIME!

In [249]:
df

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.000000e+00,0.000000e+00,0.000000e+00,3.909952e-07,0.000000e+00,0.000000e+00,0.000000e+00,1.680003e-07,8.420257e-07,0,...,0,0.000001,0.000000,0.000000e+00,0,0,0,7.809873e-07,8.690282e-07,0
1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.410031e-07,0.000000e+00,0.000000e+00,0.000000e+00,0,...,0,0.000000,0.000001,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0
2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,...,0,0.000000,0.000000,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0
3,5.370027e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,...,0,0.000000,0.000000,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0
4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,...,0,0.000000,0.000000,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0
5,0.000000e+00,5.599908e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,...,0,0.000000,0.000000,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0
6,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,...,0,0.000000,0.000000,0.000000e+00,0,0,0,1.430012e-06,0.000000e+00,0
7,2.469963e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,...,0,0.000000,0.000000,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0
8,6.689863e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,...,0,0.000000,0.000000,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0
9,1.080037e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,...,0,0.000000,0.000000,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0


In [10]:
ds = [0]
for index, row in waloc.iterrows():
    if index>=1:
        ds.append(gc_dist([115.860500,-31.9505],[row[2],row[3]]))
waloc['distance to perth'] = ds
waloc.sort_values('distance to perth')
# Sattelite towns are Bullsbrook, Mundijong, Serpentine, Yanchep

,City / Town,Population,long,latt,distance to perth
0,Perth,1874578,115.860500,-31.9505,0.000000
36,Bullsbrook,1887,115.997700,-31.6554,36.320915
43,Mundijong,1484,115.984600,-32.2996,41.583503
49,Serpentine,1265,115.976800,-32.3617,48.067193
10,Yanchep,8859,115.636600,-31.5471,50.655409
32,Two Rocks,2269,115.588700,-31.4945,57.911558
56,Mandurah,80813,115.742400,-32.5361,67.127959
20,Pinjarra,3896,115.878900,-32.6270,76.324609
28,York,2548,116.768200,-31.8935,86.995882
12,Northam,6548,116.702500,-31.6420,87.749171


In [169]:
len(town_flow[0])

57

### Problem with code above

Our choice to not consider self loops means that it accidentally created a 58x57 database. To fix this, read in the created adjacency matrices again and fix it. (Should be fixed now, run it again tomorrow though when you go play soccer). Keep this down here for reference.

In [186]:
# Fixed it up in the above code, shouldn't need to run this bit if run the above again
for i in range(len(weekbyweek)):
    name = weekbyweek[i]
    df = pd.read_csv("/Volumes/HardDrive/New_Workflow/297_towns_network/Town_Flow_" + name[23:-4] + ".csv", header = None)
    listwise = df.to_numpy().tolist()
    for j in range(len(listwise)):
        listinfocus = listwise[j]
        listinfocus.insert(j,0)
    
    with open("/Volumes/HardDrive/New_Workflow/297_towns_network/Town_Flow_" + name[23:-4] + ".csv", "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerows(listwise)   

In [210]:
name = weekbyweek[0]
df = pd.read_csv("/Volumes/HardDrive/New_Workflow/297_towns_network/Town_Flow_" + name[23:-4] + ".csv", header = None)
listwise = df.to_numpy().tolist()

In [214]:
lll = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
lll = lll.to_numpy().tolist()

In [215]:
lll

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]

In [216]:
for j in range(len(lll)):
    ll = lll[j]
    ll.insert(j,0)

In [218]:
len(lll)

4

In [ ]:
## Swallowing up sattelite towns

In [153]:
ds = [0]
for index, row in waloc.iterrows():
    if index>=1:
        ds.append(gc_dist([115.860500,-31.9505],[row[2],row[3]]))
waloc['distance to perth'] = ds
waloc.sort_values('distance to perth')[:10]
# Sattelite towns are Bullsbrook, Mundijong, Serpentine, Yanchep

,City / Town,Population,long,latt,distance to perth
0,Perth,1874578,115.8605,-31.9505,0.000000
36,Bullsbrook,1887,115.9977,-31.6554,36.320915
43,Mundijong,1484,115.9846,-32.2996,41.583503
49,Serpentine,1265,115.9768,-32.3617,48.067193
10,Yanchep,8859,115.6366,-31.5471,50.655409
32,Two Rocks,2269,115.5887,-31.4945,57.911558
56,Mandurah,80813,115.7424,-32.5361,67.127959
20,Pinjarra,3896,115.8789,-32.6270,76.324609
28,York,2548,116.7682,-31.8935,86.995882
12,Northam,6548,116.7025,-31.6420,87.749171


In [ ]:
# So take the adjacency matrices for 58 town model, if their index is that of a sattelite town add it to Perth

In [193]:
name = weekbyweek[0]
df = pd.read_csv("/Volumes/HardDrive/New_Workflow/58_towns_network/Town_Flow_" + name[23:-4] + ".csv", header = None)

In [194]:
df

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,2.539996e-07,0.000000e+00,0.0,0.0,0.0,...,0.0,0.000000,5.199900e-07,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.0
1,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.0
2,2.479933e-07,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.0
3,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.0
4,0.000000e+00,1.759968e-07,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.0
5,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,9.579781e-07,0.000000,0.0
6,1.420055e-07,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.0
7,4.480098e-07,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.0
8,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.0
9,1.519599e-05,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.0


In [195]:
llist = [3,2,4]

llist.insert(1,0)

In [196]:
llist

[3, 0, 2, 4]

In [104]:
df = pd.read_csv('/Volumes/HardDrive/weekbyweek/'+weekbyweek[50],header = None, delimiter = '\t')

In [101]:
df.to_csv('/Volumes/HardDrive/weekbyweek/'+weekbyweek[50], header = False, index = False)

In [ ]:
df = pd.read_csv('/Volumes/HardDrive/weekbyweek/'+weekbyweek[50],header = None)

In [105]:
df

,0,1,2,3,4,5,6,7
0,2021-01-24_2021-01-30,3568dad,35.103000,128.851910,3568c29,35.092823,128.977230,-15.0603
1,2021-01-24_2021-01-30,40d2f8b,49.247444,30.096780,40d2f8b,49.247444,30.096780,-13.1842
2,2021-01-24_2021-01-30,0099abb,-22.417353,-43.126260,009907f,-22.493580,-43.184402,-15.9441
3,2021-01-24_2021-01-30,8429cf1,21.877878,-102.656075,8429cef,21.876093,-102.679050,-15.9358
4,2021-01-24_2021-01-30,397c5cf,23.189340,77.343925,397c43d,23.198630,77.458725,-15.9358
...,...,...,...,...,...,...,...,...
1048571,2021-01-24_2021-01-30,5495489,45.228428,-123.179140,549548f,45.220085,-123.204650,-12.5967
1048572,2021-01-24_2021-01-30,6000549,35.027798,135.572370,600054f,35.010060,135.572370,-13.9652
1048573,2021-01-24_2021-01-30,486e1cb,51.482456,-3.181969,4.86E+195,51.586710,-3.218967,-15.9111
1048574,2021-01-24_2021-01-30,391bc2f,31.810684,75.189560,391be43,31.933098,75.096150,-15.6670
